In [3]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
import numpy as np
import cv2
import os
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt
from numpy import load


In [4]:
label = "6_way_label"
model_name = "model_6_way.h5"
num_of_classes = 6
epochs = 20
input_feature_size = 1536
train_test_ratio = 0.1
batch_size = 1


In [5]:
img_features = load('new_img_features.npy')
bert_embeddings = load('new_bert_embeddings.npy')
print(img_features.shape)
print(bert_embeddings.shape)

(8319, 768)
(8319, 768)


In [6]:
features = pd.read_csv("data_without_features.csv")
features.shape

(8319, 5)

In [7]:
features.head()

,Unnamed: 0,img_path,2_way_label,3_way_label,6_way_label
0,0,D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...,1.0,0.0,0.0
1,1,D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...,0.0,2.0,1.0
2,2,D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...,0.0,2.0,4.0
3,3,D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...,1.0,0.0,0.0
4,4,D:\UB\2nd_sem\CSE_676_DL\Project\DL_work\Code\...,0.0,2.0,4.0


In [8]:
merged_features = np.concatenate((img_features, bert_embeddings), axis=1)
merged_features.shape

(8319, 1536)

In [9]:
X = merged_features
Y = np.array(features[label])
Y = tf.keras.utils.to_categorical(Y, num_of_classes)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=train_test_ratio, random_state=123)

In [11]:
print("x_train-",x_train.shape)
print("x_test-",x_test.shape)
print("y_train-",y_train.shape)
print("y_test-",y_test.shape)

x_train- (7487, 1536)
x_test- (832, 1536)
y_train- (7487, 6)
y_test- (832, 6)


In [12]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (7487, 1536)
7487 train samples
832 test samples


In [13]:
model = Sequential()

model.add(Dense(256, input_dim=input_feature_size,  activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_of_classes, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Model Summary-",model.summary())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               393472    
_________________________________________________________________
dense_1 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_2 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 198       
Total params: 436,902
Trainable params: 436,902
Non-trainable params: 0
________________________________________________________________

In [14]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/20
7487/7487 [==============================] - 9s 1ms/sample - loss: 0.3439 - acc: 0.8516
Epoch 2/20
7487/7487 [==============================] - 9s 1ms/sample - loss: 0.3137 - acc: 0.8681
Epoch 3/20
7487/7487 [==============================] - 9s 1ms/sample - loss: 0.3010 - acc: 0.8742
Epoch 4/20
7487/7487 [==============================] - 9s 1ms/sample - loss: 0.2923 - acc: 0.8789
Epoch 5/20
7487/7487 [==============================] - 9s 1ms/sample - loss: 0.2851 - acc: 0.8801
Epoch 6/20
7487/7487 [==============================] - 9s 1ms/sample - loss: 0.2798 - acc: 0.8831
Epoch 7/20
7487/7487 [==============================] - 9s 1ms/sample - loss: 0.2752 - acc: 0.8841
Epoch 8/20
7487/7487 [==============================] - 9s 1ms/sample - loss: 0.2697 - acc: 0.8878
Epoch 9/20
7487/7487 [==============================] - 9s 1ms/sample - loss: 0.2645 - acc: 0.8907
Epoch 10/20
748

In [15]:
score = model.evaluate(x_test, y_test)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

832/832 [==============================] - 0s 54us/sample - loss: 0.3309 - acc: 0.8708
Test loss: 0.3308791294693947
Test accuracy: 0.8707932


In [16]:
model.save(model_name)